1. https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
2. https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/
3. https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
4. https://www.tutorialspoint.com/keras/keras_regression_prediction_using_mpl.htm
5. https://machinelearningmastery.com/binary-classification-tutorial-with-the-keras-deep-learning-library/

Naudinga medžiaga:
1. https://towardsdatascience.com/airbnb-price-prediction-multilayer-perceptrons-with-tensorflows-keras-api-9096e7d0c340
2. https://machinelearningmastery.com/how-to-make-classification-and-regression-predictions-for-deep-learning-models-in-keras/

# import libraries

In [ ]:
#for all 3 types
import pandas as pd
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from scikeras.wrappers import KerasRegressor
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


# load the dataset
https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [ ]:
#two-class classification
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv'
dataframe = pd.read_csv(url, header=None)
dataframe.head()

In [ ]:
# multi-class classification
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv'
dataframe = pd.read_csv(url, header=None)
dataframe.head()

In [103]:
#regression
url='https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.data'
dataframe = pd.read_csv(url, delim_whitespace=True, header=None)
#or
#dataframe = pd.read_csv(url, header=None, delimiter=r"\s+")
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [104]:
#for all 3 types
dataframe.dtypes

0     float64
1     float64
2     float64
3       int64
4     float64
5     float64
6     float64
7     float64
8       int64
9     float64
10    float64
11    float64
12    float64
13    float64
dtype: object

In [105]:
#for all 3 types

#convert pandas dataframe to numpy ndarray - from this part you can't use pandas syntax
dataset = dataframe.values

In [106]:
#for all 3 types

# split into input (X) and output (y) variables
X = dataset[:,0:-1]
y = dataset[:,-1]

In [107]:
#for all 3 types

#https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/
# define standard scaler
scaler = StandardScaler()
# transform data
X = scaler.fit_transform(X)

In [108]:
#regression

y = y.reshape(-1, 1) #sometimes don't need to use it
y = scaler.fit_transform(y)

In [ ]:
# multi-class classification

## encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
#print(encoded_Y)

## convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
y = dummy_y
#print(y)

# define the keras model

In [ ]:
#two-class classification

## define baseline model
def baseline_model():

	# create model
    model = Sequential()
    model.add(Dense(12, input_shape=(X.shape[1],), activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model
    

In [ ]:
# multi-class classification

## define baseline model
def baseline_model():

	# create model
	model = Sequential()
	model.add(Dense(8, input_shape=(X.shape[1],), activation='relu'))
	model.add(Dense(3, activation='softmax'))

	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

	return model

In [109]:
#regression
#https://machinelearningmastery.com/regression-metrics-for-machine-learning/

## define baseline model
def baseline_model():

	# create model
	model = Sequential()
	model.add(Dense(20, input_shape=(X.shape[1],), activation='relu'))
	model.add(Dense(1, activation='linear'))

	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')

	return model

# run models

In [ ]:
#two-class classifcation

baseline_model().summary()

estimator = KerasClassifier(model=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)

results = cross_val_score(estimator, X, y, cv=kfold)
print("Baseline model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
# multi-class classification 

baseline_model().summary()

estimator = KerasClassifier(model=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)

results = cross_val_score(estimator, X, y, cv=kfold)
print("Baseline model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


In [110]:
#regression

baseline_model().summary()

estimator = KerasRegressor(model=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)

#https://stephenallwright.com/cross_val_score-sklearn/
#https://scikit-learn.org/stable/modules/model_evaluation.html
#we cannot calculate accuracy for a regression model
results = cross_val_score(estimator, X, y, cv=kfold, scoring='neg_root_mean_squared_error')
print("Baseline model: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Model: "sequential_171"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_363 (Dense)           (None, 20)                280       
                                                                 
 dense_364 (Dense)           (None, 1)                 21        
                                                                 
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________
Baseline model: -0.38 (0.06) MSE


In [ ]:
results